# HTML文档处理流程
## 任务：
1. 复制HTML文件，命名为title内容（去除特殊字符）
2. 处理HTML内容
3. 转换为markdown格式
4. 提取p标签内容并按句号分割


In [31]:
from bs4 import BeautifulSoup
import re
import html2text
import shutil


In [32]:
# 读取原始HTML文件
input_file = 'output_document.html'

with open(input_file, 'r', encoding='utf-8') as f:
    html_content = f.read()

# 解析HTML
soup = BeautifulSoup(html_content, 'html.parser')

# 获取title内容
title = soup.find('title').text
print(f"原始标题: {title}")

# 去除特殊字符
clean_title = re.sub(r'[\\/:*?"<>|]', '', title)
print(f"清理后的标题: {clean_title}")


原始标题: 坏死性软组织感染临床诊治急诊专家共识
清理后的标题: 坏死性软组织感染临床诊治急诊专家共识


In [33]:
# 步骤1: 复制文件，使用title作为文件名
copy_filename = f"{clean_title}.html"
shutil.copy(input_file, copy_filename)
print(f"已复制文件到: {copy_filename}")


已复制文件到: 坏死性软组织感染临床诊治急诊专家共识.html


In [34]:
# 步骤2: 处理原文件内容
# 重新解析用于处理的HTML
soup_processing = BeautifulSoup(html_content, 'html.parser')

# 2.1 找到包含"DOI:"的p标签，删除该标签及其之前的所有p标签
all_p_tags = soup_processing.find_all('p')
doi_index = -1

for i, p in enumerate(all_p_tags):
    if 'DOI:' in p.get_text():
        doi_index = i
        print(f"找到DOI标签，位置: {i}，内容: {p.get_text()[:50]}")
        break

# 删除DOI及之前的所有p标签
if doi_index >= 0:
    for i in range(doi_index + 1):
        all_p_tags[i].decompose()
    print(f"已删除前{doi_index + 1}个p标签")


找到DOI标签，位置: 6，内容: DOI: 10.3760/cma.j.issn.1671-0282.2023.11.007
已删除前7个p标签


In [35]:
# 2.2 从文件末尾向前查找包含"参考文献"的内容，删除该内容之后的所有部分
# 查找包含"参考文献"的标签（处理"参 考 文 献"这种带空格的情况）
reference_tag = None
for tag in soup_processing.find_all(['h1', 'h2', 'h3', 'p']):
    # 去除所有空白字符后检查是否包含"参考文献"
    clean_text = re.sub(r'\s+', '', tag.get_text())
    if '参考文献' in clean_text:
        reference_tag = tag
        print(f"找到参考文献标签: {tag.name}, 原始内容: {tag.get_text()}")
        break

# 删除参考文献标签及之后的所有内容
if reference_tag:
    # 获取该标签之后的所有兄弟标签并删除
    for sibling in list(reference_tag.next_siblings):
        if hasattr(sibling, 'decompose'):
            sibling.decompose()
    # 删除参考文献标签本身
    reference_tag.decompose()
    print("已删除参考文献及之后的内容")
else:
    print("警告：未找到参考文献标签")


找到参考文献标签: h2, 原始内容: 参 考	 文	 献
已删除参考文献及之后的内容


In [36]:
# 2.3 删除所有table标签
tables = soup_processing.find_all('table')
print(f"找到{len(tables)}个table标签")
for table in tables:
    table.decompose()
print("已删除所有table标签")


找到5个table标签
已删除所有table标签


In [37]:
# 2.4 去除所有p标签内部的空格、换行符以及[]符号及其内容
p_tags = soup_processing.find_all('p')
for p in p_tags:
    # 获取文本内容
    text = p.get_text()
    # 先去除[]及其中的内容
    text = re.sub(r'\[.*?\]', '', text)
    # 再去除空格和换行符
    cleaned_text = re.sub(r'\s+', '', text)
    # 替换p标签的内容
    p.string = cleaned_text

print(f"已处理{len(p_tags)}个p标签，去除空格、换行符和[]符号")


已处理73个p标签，去除空格、换行符和[]符号


In [38]:
# 步骤3: 将处理后的HTML转换为markdown格式
processed_html = str(soup_processing)

# 使用html2text转换
h = html2text.HTML2Text()
h.ignore_links = False
h.ignore_images = False
h.body_width = 0  # 不限制宽度

markdown_content = h.handle(processed_html)

# 保存为ragt.md
with open('ragt.md', 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print("已保存markdown文件: ragt.md")
print(f"Markdown内容长度: {len(markdown_content)} 字符")


已保存markdown文件: ragt.md
Markdown内容长度: 11814 字符


In [39]:
# 步骤4: 提取所有p标签内容，每十个p标签拼接成一行
p_tags = soup_processing.find_all('p')

# 提取所有p标签的文本内容
p_texts = []
for p in p_tags:
    text = p.get_text()
    if text.strip():  # 只处理非空文本
        # 去除[]及其中的内容（如果之前步骤有遗漏）
        text = re.sub(r'\[.*?\]', '', text)
        text = text.strip()
        if text:
            p_texts.append(text)

# 保存为ner.txt，每十个p标签拼接成一行
with open('ner.txt', 'w', encoding='utf-8') as f:
    for i in range(0, len(p_texts), 10):
        # 获取当前批次的p标签内容（最多10个）
        batch = p_texts[i:i+10]
        # 用空格连接这些p标签内容，写入一行
        f.write(' '.join(batch) + '\n')

print(f"已保存ner.txt文件，共{len(p_texts)}个p标签，每10个p标签一行")
print(f"总共{(len(p_texts) + 9) // 10}行")
print("\n前3个p标签内容示例:")
for i, text in enumerate(p_texts[:3]):
    print(f"{i+1}. {text[:80]}..." if len(text) > 80 else f"{i+1}. {text}")


已保存ner.txt文件，共73个p标签，每10个p标签一行
总共8行

前3个p标签内容示例:
1. 坏死性软组织感染(necrotizingsofttissueinfections，NSTIs)是由病原微生物感染导致皮下组织、筋膜或（和）肌肉坏死的一类疾病。N...
2. 本专家共识以软组织感染、坏死性软组织感染、坏死性筋膜炎为中文检索词，以Necrotizingfasciitis、Softtissueinfections、Nec...
3. 据报道，每年皮肤软组织感染的急诊就诊人数约540~840万，其中12%~40%患者需住院治疗，0.7%需收住监护室。NSTIs发病率在(1.1~15.5)/10...


In [40]:
# 总结输出
print("\n=== 处理完成 ===")
print(f"1. 已复制原文件为: {copy_filename}")
print(f"2. 已删除DOI之前的{doi_index + 1}个p标签")
print(f"3. 已删除参考文献及之后的内容")
print(f"4. 已删除{len(tables)}个table标签")
print(f"5. 已去除所有p标签内的空格、换行符和[]符号")
print(f"6. 已保存markdown文件: ragt.md")
print(f"7. 已保存ner.txt文件（共{len(p_texts)}个p标签，每10个p标签一行）")



=== 处理完成 ===
1. 已复制原文件为: 坏死性软组织感染临床诊治急诊专家共识.html
2. 已删除DOI之前的7个p标签
3. 已删除参考文献及之后的内容
4. 已删除5个table标签
5. 已去除所有p标签内的空格、换行符和[]符号
6. 已保存markdown文件: ragt.md
7. 已保存ner.txt文件（共73个p标签，每10个p标签一行）
